**This notebook is the executable version of lab note 3.
It answers the following questions:**

Finally, we answer 4 items:

    1. Are successful creators more connected to high-outdegree users than do unsuccessful creators?
    2. Are mavens more connected to successful creators than to unsuccessful creators?
    3. Do successful creators send more non-follow actions towards mavens than to zombies?
    4. Do successful creators send more non-follow actions towards mavens than to stars?

    

In [1]:
#Run parameters
#used to control every run. Can be user to perfom sensitivity checks
path_dir = r"/Users/../Volumes/Raw/"

low_success = 0.5 #below the median: unsuccessful
high_success = 0.9 #top 10% creators with more followers are deemed successful

low_user_outdegree = 0.25 
high_user_outdegree = 0.75
low_user_activity = 0.25 
high_user_activity = 0.75 

activity_filter = 0
days_delta = 7

In [2]:
import sys  
import pickle
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
import src.utils
from collections import Counter
from src.utils import import_dta, import_tracks_dta,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob, stripplot_prob,\
reaction_probability, follower_list, filter_quantile, sample_creators_music,\
gen_outbound_creators
import numpy as np
import datetime
import pandas as pd
from tqdm import tqdm
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import os
from statsmodels.stats.proportion import proportions_ztest
from datetime import datetime, timedelta


In [3]:
def process_date(date):
    '''convert date format like '2013-w09' to '2013-03-04', i.e. the first day of that week'''
    year = date[0:4]
    week = date[6:]
    day = "1"
    date = "{}-{}-1".format(year, week)
    dt = datetime.strptime(date, "%Y-%W-%w")
    return dt

# Data Imports

We start by importing the raw data.  `follows_sent`, `comments_sent`, `shares_sent`, `likes_sent` and `messages_sent` contains data pn the promotional activities that the 35k users tracked in the dataset directed to other users. It includes the `user_id`, the `fan_id` and the `date_sent` which identifies the date when the prom. activity was sent. `users_info_1st` shows the type of user (creator or non-creator, which is identified by a blank) and the date the user entered the platform, for every user that sent or received prom. activities from any of the 35k users tracked in this dataset, while `users_info` contains the same information, but pertaining to the 35k users themselves.

`follows_received` contains information on the follows received by the 35k users and will be used to generate the successful/unsuccessful groups of content creators.

In [4]:
#affiliations :follows
#favoritings :likes

#used in filtering:
path_dir = r"/Users/../Volumes/Raw/"
tracks = import_tracks_dta(path_dir, "12sample_tracks.dta");

#these are the actions sent to 
follows_sent = import_dta(path_dir, "12sample_affiliations_sent.dta");
comments_sent = import_dta(path_dir, "12sample_comments_made.dta");
shares_sent = import_dta(path_dir, "12sample_reposts_made.dta");
likes_sent = import_dta(path_dir, "12sample_favoritings_made.dta");
messages_sent = import_dta(path_dir, "12sample_messages_sent.dta");

#Used to track information on the 1st degree connections
user_info_1st = import_dta(path_dir, "12sample_1st_deg_user_infos.dta");
user_info_1st.columns = ['user_id', 'type', 'entered_platform'];
user_info = import_dta(path_dir, "12sample_user_infos.dta");

#Used to compute creator's success measure
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta");

%%%%%%%%%% 12sample_tracks.dta %%%%%%%%%%
(56262, 7)
%%%%%%%%%% 12sample_affiliations_sent.dta %%%%%%%%%%
(800913, 3)
%%%%%%%%%% 12sample_comments_made.dta %%%%%%%%%%
(29258, 4)
%%%%%%%%%% 12sample_reposts_made.dta %%%%%%%%%%
(179329, 4)
%%%%%%%%%% 12sample_favoritings_made.dta %%%%%%%%%%
(527701, 4)
%%%%%%%%%% 12sample_messages_sent.dta %%%%%%%%%%
(11091, 3)
%%%%%%%%%% 12sample_1st_deg_user_infos.dta %%%%%%%%%%
(670746, 3)
%%%%%%%%%% 12sample_user_infos.dta %%%%%%%%%%
(35000, 3)
%%%%%%%%%% 12sample_affiliations_received.dta %%%%%%%%%%
(432503, 3)


Indegree and outdegree information.

The function below import the outdegree dataset. Because the raw version of those dataset are too large to be processed in memory, we preprocessed them in a separate script.

In [5]:
# Aggregates preprocessed outdegree of 1st degree users
def import_outdegree(path='/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/'):
    d = {}
    for i in range(6):
       d[str(i)] = pd.read_pickle(os.path.join(path,'{}.pkl'.format(i))) 
       d[str(i)]['created_at'] =  pd.to_datetime(d[str(i)]['created_at'])
       d[str(i)]['created_at'] = pd.to_datetime(d[str(i)]['created_at']).dt.floor('d')
       d[str(i)] = d[str(i)].groupby(['sender_id', 'created_at'], as_index = False).size() 
    
    data_outdegree = pd.concat([d['0'], d['1'], d['2'], d['3'], d['4'], d['5']])
    #data_outdegree.set_index('created_at', inplace = True)
    return data_outdegree

In [ ]:
data_outdegree = import_outdegree()

In [ ]:
#data_outdegree = data_outdegree.groupby(['sender_id','created_at'], as_index = False).size()

# Preprocessing

## Creator ids, successful and unsucessful creators

Next, we define three lists of ids: one with the ids from the content creators, according to the `users_info` table, one with the ids of successful creators and the last one with the ids of the unsuccessful ones.

Let's start with a list of the id of creators. We also create a dataset with containing information on creators only.

In [ ]:
mask = (tracks.track_available == 1) & (tracks.public == 't')
creator_ids = tracks[mask].user_id.unique()

creators = tracks[(tracks.track_available == 1) & (tracks.public == 't')]

#mask = user_info.type == 'creator'
#creator_ids = user_info[mask].user_id.unique()

#creators = user_info[user_info.type == 'creator']

## Putting together a dataset with the promotional activities made by content creators.

The function `gen_actions_sent_df` creates a dataframe with all the promotional activities that content creators sent to users.

In [ ]:
def gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent, messages_sent, creator_ids = creator_ids):
    '''
    Creates dataframe containing the actions that content creators send to users.
        Attributes:
                    follows_sent:  dataframe with the follows sent by the 35k users.
                    shares_sent:   dataframe with the shares sent by the 35k users.
                    likes_sent:    dataframe with the likes sent by the 35k users.
                    comments_sent: dataframe with the comments sent by the 35k users.
                    messages_sent: dataframe with the messages sent by the 35k users.
                    creator_ids:   list with content creator ids. If not none, is used to
                                   filter out activities from non creators.
    '''
    
    follows_sent['outbound_activity'] = 'follow'
    follows_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'song_id' in shares_sent.columns:
        shares_sent.drop(columns=["song_id"])
    shares_sent = shares_sent[['reposter_id', "owner_id", 'created_at']]
    shares_sent['outbound_activity'] = 'share'
    shares_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in likes_sent.columns:
        likes_sent.drop(columns=["track_id"], inplace=True)
    likes_sent['outbound_activity'] = 'like'
    likes_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    if 'track_id' in comments_sent.columns:
        comments_sent.drop(columns=["track_id"], inplace=True)
    comments_sent['outbound_activity'] = 'comment'
    comments_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

    messages_sent["outbound_activity"] = 'message'
    messages_sent.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']
    df = pd.concat([follows_sent, shares_sent, likes_sent, comments_sent, messages_sent])


    if type(creator_ids) == numpy.ndarray:
        df = df[df['user_id'].isin(creator_ids)]
        
    df['week_yr'] = df.date_sent.dt.strftime('%Y-w%U')
    df = df.loc[df['user_id'] != df['fan_id'],:]

    return df

In [ ]:
actions_sent = gen_actions_sent_df(follows_sent, shares_sent, likes_sent, comments_sent,
                                     messages_sent, creator_ids = None)
actions_sent = actions_sent.loc[actions_sent.user_id.isin(creators.user_id.unique())]

In [ ]:
active_users_ids = actions_sent.groupby('user_id', as_index = False).size()
mask = active_users_ids['size']>= activity_filter
active_users_ids = active_users_ids[mask].user_id.unique()

In [ ]:
def successful_creators_followers(follows_received, base_date = datetime(2016, 5, 30, 0, 0), perc1 = None, perc2 = None, subset_creators = None):
    '''Classifies content creators in successful or unsuccessfull
        Arguments:
                    follows_received: dataframe containing the follows received by content creators
                    base date:        date, in datetime.datetime(YYYY, M, DD, H, M) format, in which the number 
                                      of followers per creator is calculated.
                    perc1:            the threshold used to classify unsuccessful content creators. Creator having 
                                      total followers below the number dictated by this threshold, at the base date,
                                      are classified as unsuccessful 
                    perc2:            the threshold used to classify successful content creators. Creator having 
                                      total followers above the number dictated by this threshold, at the base date,
                                      are classified as successful
                    subset_creators:  a pd.DataFrame containing the creators. If is it available, it will be used to 
                                      filter out non creators and to make sure creators with 0 followers are part of
                                      the resulting dataset.
        
    '''
    print(base_date)

    if 'inbound_activity' not in follows_received.columns:
        follows_received.columns = ['fan_id', 'user_id', 'date_sent']

    mask = (follows_received['date_sent'] < base_date)

    df = follows_received[mask].groupby('user_id', as_index=False).agg({'fan_id': pd.Series.nunique})
    df.columns = ['user_id', 'followers']

    
    if type(subset_creators) == pd.DataFrame:
        print('subsetting...')
        df.set_index('user_id', inplace = True)
        df = df.reindex(subset_creators.user_id.unique())
        df.fillna(0, inplace = True)
        df.reset_index(inplace = True)
        df.columns = ['user_id', 'followers']
        
    mask = df.user_id.isin(active_users_ids)
    df = df[mask]

    low = np.quantile(df.followers, perc1)
    high = np.quantile(df.followers, perc2)

    print("High influencer boundary: {}".format(high))
    print("Low influencer boundary: {}".format(low))

    mask = (df["followers"] <= low) | (df["followers"] >= high)
    
    unsuccessful_creator_ids = df.loc[df["followers"] <= low].user_id.unique()
    successful_creator_ids = df.loc[df["followers"] >= high].user_id.unique()

    return unsuccessful_creator_ids, successful_creator_ids


In [ ]:
unsuccessful_ids, successful_ids = successful_creators_followers(follows_received, 
                                                        perc1 = low_success, perc2 = high_success, subset_creators = creators)


In [ ]:
print(len(unsuccessful_ids))
print(len(successful_ids))

In [ ]:
creators.user_id.nunique()

## Filter only actions that were sent to non-fans

We merge the `actions_sent` dataset with a table containing the date each fan started following the creator.

In [ ]:
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follows_received[["fan_id", "user_id", "date_sent"]]
followers.columns = ["fan_id", "user_id", "follower_since"]

actions_sent = actions_sent.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')

Since we are interested in acquisition campaings, we need to produce a dataset that exclude actions targetting fans.
We do that using filters based on the date of the action and the date that the user became a fan of the content creator. The resulting dataframe is named `actions_sent_non_fans`.

We then filter only actions that happened before the user follows the content creator.

In [ ]:
mask = (actions_sent.date_sent < actions_sent.follower_since) | (actions_sent.follower_since.isnull())
actions_sent_non_fans =  actions_sent[mask]
actions_sent_non_fans['week_yr_date'] = actions_sent_non_fans.week_yr.apply(lambda x: process_date(x))

## Outdegree level

Originally, we only have outdegree information on users that follow at least one user. The function below inputs an outdegree of 0 to users that are following anyone.

In [ ]:
#CHANGE DOCUMENTATION
def outdegree_level(data, date, user_info = user_info_1st):
    
    '''
    This function returns the membership table at date equals `date`. Every user that interacted with the 35k tracked 
    users and entered the platform before `date` is present in the table, even if it has indegree 0.
    arguments:
              data:           the indegree dataset.
              user_info:      the dataset containing all the users that interacted with the 35k users tracked.
    '''
    
    data = data[data.created_at.dt.floor('d')<=date]
    data = data.groupby('sender_id').agg({'size':'sum'}).compute()
    
    #merge with user info to obtain users that are not followed by anyone at the current date
    data = user_info_1st.merge(data, left_on = 'user_id', right_on = 'sender_id', how= 'outer')
    data.loc[data['size'].isnull(), 'size'] = 0
    data = data[['user_id', 'size', 'entered_platform']].set_index('user_id')
    
    #filter out users that didnt exist in the current date
    mask = data['entered_platform'].dt.floor('d') <= date
    data = data.loc[mask]
    
    mask = (data['size']>0)
    data.loc[~mask, 'size'] = 0 
    
    return data

In [ ]:
#dask is a python api with objects optimized for big data (user directed acyclic graphs). 
dask_outdegree = dd.from_pandas(data_outdegree, npartitions = 3)

In [ ]:
last_day =  max(actions_sent.date_sent.dt.floor('d').unique())
user_outdegree = outdegree_level(dask_outdegree, last_day, user_info = user_info_1st)

Now we classify the creator in successful and unsuccessful, according to the threshold defined in the beggining of this notebook.

In [ ]:
def user_outdegree_level(oudegree_data, perc1 = None, perc2 = None):
    '''Classifies content creators in successful or unsuccessfull
        Arguments:
                    oudegree_data:    dataframe containing the fans followers at 
                    perc1:            the threshold used to classify unsuccessful content creators. Creator having 
                                      total followers below the number dictated by this threshold, at the base date,
                                      are classified as unsuccessful 
                    perc2:            the threshold used to classify successful content creators. Creator having 
                                      total followers above the number dictated by this threshold, at the base date,
                                      are classified as successful
    '''

    df = oudegree_data.reset_index().iloc[:,:2]
    df.columns = ['user_id', 'followers']

    low = np.quantile(df.followers, perc1)
    high = np.quantile(df.followers, perc2)

    print("High outdegree boundary: {}".format(high))
    print("Low outdegree boundary: {}".format(low))
    
    low_outdegree_ids = df.loc[df["followers"] <= low].user_id.unique()
    high_outdegree_ids = df.loc[df["followers"] >= high].user_id.unique()
    
    
    df['outdegree_level'] = df.user_id.apply(
        lambda x: 'high' if x in high_outdegree_ids else ('low' if x in low_outdegree_ids else None))

    
    return df, low_outdegree_ids, high_outdegree_ids

In [ ]:
user_outdegree_df, low_outdegree_ids, high_outdegree_ids = user_outdegree_level(user_outdegree,
                perc1 = low_user_outdegree, perc2 = high_user_outdegree)

In the cell below, we create a list with unique ids from users that appear in the oudegree level table. This will later be uses to construct a flow-chart indicating how we lose data based on filters and operations.

In [ ]:
receiver_ids = user_outdegree.index.unique()

## Non-follow Actions level

The activity level is defined as the number of actions performed by users. It is important to notice that we only observe actions targeting the 35k users that joined in march 2012. We consider this measure a proxy for the real activity level.

Let's begin by creating a dataset with all action received by those 35k users.

In [ ]:
comments_received_35k = import_dta(path_dir, "12sample_comments_received.dta");
shares_received_35k = import_dta(path_dir, "12sample_reposts_received.dta");
likes_received_35k = import_dta(path_dir, "12sample_favoritings_received.dta");
messages_received_35k = import_dta(path_dir, "12sample_messages_received.dta");

if 'song_id' in shares_received_35k:
        shares_received_35k.drop(columns=["song_id"])
shares_received_35k = shares_received_35k[['reposter_id', "owner_id", 'created_at']]
shares_received_35k['outbound_activity'] = 'share'
shares_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in likes_received_35k:
        likes_received_35k = likes_received_35k.drop(columns=["track_id"])
likes_received_35k['outbound_activity'] = 'like'
likes_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in comments_received_35k:
        comments_received_35k = comments_received_35k.drop(columns=["track_id"])
comments_received_35k['outbound_activity'] = 'comment'
comments_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

messages_received_35k["outbound_activity"] = 'message'
messages_received_35k.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

user_activity_data_35k = pd.concat([shares_received_35k, likes_received_35k, comments_received_35k, messages_received_35k])

In [ ]:
path_dir_2 = r'/Users/../Volumes/Alter_outbound_activities/'

comments_received_c = import_dta(path_dir_2, "12sample_1st_deg_comments_made.dta");
shares_received_c = import_dta(path_dir_2, "12sample_1st_deg_reposts_made.dta");
likes_received_c = import_dta(path_dir_2, "12sample_1st_deg_favoritings_made.dta");
messages_received_c = import_dta(path_dir_2, "12sample_1st_deg_messages_sent.dta");

In [ ]:
if 'song_id' in shares_received_c:
        shares_received_c.drop(columns=["song_id"])
shares_received_c = shares_received_c[['reposter_id', "owner_id", 'created_at']]
shares_received_c['inbound_activity'] = 'share'
shares_received_c.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in likes_received_c:
        likes_received_c = likes_received_c.drop(columns=["track_id"])
likes_received_c['inbound_activity'] = 'like'
likes_received_c.columns = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

if 'track_id' in comments_received_c:
        comments_received_c = comments_received_c.drop(columns=["track_id"])
comments_received_c['inbound_activity'] = 'comment'
comments_received_c.columns_c = ['fan_id', 'user_id', 'date_sent', 'inbound_activity']

messages_received_c["outbound_activity"] = 'message'
messages_received_c.columns = ['user_id', 'fan_id', 'date_sent', 'inbound_activity']

In [ ]:
user_activity_data_c = pd.concat([shares_received_c, likes_received_c, comments_received_c, messages_received_c])

In [ ]:
def user_activity_level(user_activity_data, receiver_ids, perc1 = None, perc2 = None):
    '''Classifies users based on the amount of non-follow activities
        Arguments:
                    user_activity_data:   dataframe containing the user activities 
                    base date:             date, in datetime.datetime(YYYY, M, DD, H, M) format, in which the number 
                                           of followers per creator is calculated.
                    perc1:                 the threshold used to classify unsuccessful content creators. Creator having 
                                           total followers below the number dictated by this threshold, at the base date,
                                           are classified as unsuccessful 
                    perc2:                 the threshold used to classify successful content creators. Creator having 
                                           total followers above the number dictated by this threshold, at the base date,
                                           are classified as successful
    '''

    df = user_activity_data.groupby('fan_id', as_index = True).size()
    df = df.reindex(receiver_ids)
    print(df.shape)
    df = df.reset_index()
    df.columns = ['user_id', 'activities_performed']
    
    df.loc[df.activities_performed.isna(), 'activities_performed'] = 0
    #classification should happen after this

    low = np.quantile(df.activities_performed, perc1)
    high = np.quantile(df.activities_performed, perc2)

    print("High activity boundary: {}".format(high))
    print("Low activity boundary: {}".format(low))
    
    low_activity_ids = df.loc[df["activities_performed"] <= low].user_id.unique()
    high_activity_ids = df.loc[df["activities_performed"] > high].user_id.unique()
    
    df['activity_level'] = df.user_id.apply(
    lambda x: 'high' if x in high_activity_ids else ('low' if x in low_activity_ids else None))

    return df, low_activity_ids, high_activity_ids

In [ ]:
activity_level, low_activity_ids, high_activity_ids = user_activity_level(user_activity_data_c, receiver_ids, 
                perc1 = low_user_activity, perc2 = high_user_activity)

Once more we create an object containing the unique ids of users in the resulting dataset. This will be used in a flow-chart, as explained.

In [ ]:
activity_ids = np.append(low_activity_ids, high_activity_ids)

# Priori response probabilities

In [ ]:
table_data = user_outdegree_df.merge(activity_level, left_on = 'user_id', right_on = 'user_id', how = 'inner')
table_data.groupby(['outdegree_level', 'activity_level']).size()
table_data['Type'] = table_data.apply(lambda x: 
    'f_a' if (x.outdegree_level == 'low') & (x.activity_level == 'high')
     else ('Hermit' if (x.outdegree_level == 'low') & (x.activity_level == 'low')
     else ('Observer' if (x.outdegree_level == 'high') & (x.activity_level == 'low')
     else ('w_a' if (x.outdegree_level == 'high') & (x.activity_level == 'high')
     else 'other'))), axis=1)

hermit_ids = table_data.loc[table_data.Type =='Hermit'].user_id.unique()
w_a_ids = table_data.loc[table_data.Type =='w_a'].user_id.unique()
f_a_ids = table_data.loc[table_data.Type =='f_a'].user_id.unique()
observer_ids = table_data.loc[table_data.Type =='Observer'].user_id.unique()

In [ ]:
actions_sent['user_type'] = actions_sent.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
actions_sent['creator_type'] = actions_sent.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

In [ ]:
actions_sent_non_fans['user_type'] = actions_sent_non_fans.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
actions_sent_non_fans['creator_type'] = actions_sent_non_fans.user_id.apply(
                                            lambda x: 'successful' if x in successful_ids else 
                                            ('unsuccessful' if x in unsuccessful_ids else 'other'))

In [ ]:
import datetime
# Target Creation
delta = timedelta(days = days_delta)
mask = (actions_sent_non_fans['follower_since'] <= (actions_sent_non_fans['date_sent'] + delta))

response_df = actions_sent_non_fans.copy()
response_df.loc[mask, 'reward'] = 1
mask = response_df['reward'].isnull()
response_df.loc[mask, 'reward'] = 0

In [ ]:
attempts = response_df.groupby(['creator_type', 'user_type'], as_index = False).size()
rewards = response_df.groupby(['creator_type', 'user_type'], as_index = False).agg({'reward' : 'sum'})

priori_prob_df = rewards.merge(attempts)
priori_prob_df['P_resp_prob'] = priori_prob_df['reward']/priori_prob_df['size']
priori_prob_df 

# Indirect Return from Friends

In [ ]:
# the files below contain the followers for targets that:
#1. shared creator content, 
#2. performed any action towards a creator, or
#3. followed a creator
#it was necessary do split the data because it was too large to in memory.
#on a future interaction it can be made better
import pickle as pkl
share_follower_lists = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists.pkl')
action_follower_lists = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists_2.pkl')
follow_follower_lists = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists_3.pkl')

In [ ]:
#different approach, probably also good
def compute_indirect_return(df, follower_list):    
    follower_lists = follower_list
    data = df
    data = data.loc[data.user_id.isin(creators.user_id)]

    data = data.loc[data.fan_id.isin(follower_lists.index.unique())]
    data['expec_followers'] = np.nan
    data.sort_values(['user_id', 'fan_id'], inplace = True)
    
    df.date_sent = df.date_sent.dt.normalize()

    for user_id in tqdm(data.user_id.unique()):
        df_j = df.loc[df.user_id == user_id]
        for fan_id in data.loc[data.user_id == user_id].fan_id.unique():
            fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]
            data.loc[(data.user_id == user_id)&
                                      (data.fan_id == fan_id),'expec_followers'] =\
            data.loc[(data.user_id == user_id)&
                                      (data.fan_id == fan_id)].date_sent.apply(
                lambda x : 
                np.sum(np.in1d(
                               df_j.loc[(x <= df_j.date_sent) & 
                                                      (df_j.date_sent<= x + 
                                                       timedelta(days=days_delta))
                                                     ].values,
                    fan_followers)
            ))                   
    return data

In [ ]:
def compute_stats(df):
    df['user_type'] = df.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                              ('Hermit' if x in hermit_ids else
                              ('Observer' if x in observer_ids else
                              ('w_a' if x in w_a_ids else 'other'))))

    ## classify content creators

    df['creator_type'] = df.user_id.apply(lambda x: 'successful' if x in successful_ids else 
                                                ('unsuccessful' if x in unsuccessful_ids else 'other'))
    attempts = df.groupby(['creator_type', 'user_type'], as_index = False).size()
    desc = df.groupby(['creator_type', 'user_type'], as_index = False).agg(expected_followers=('expec_followers', 'mean'),
                                   std_expected_followers=('expec_followers', 'std'),
                                   max_followers=('expec_followers', 'max'))

    return attempts.merge(desc)

In [ ]:
expec_followers_given_follow_df = compute_indirect_return(follows_received, follow_follower_lists)
indirect_return_follows = compute_stats(expec_followers_given_follow_df)

In [ ]:
indirect_return_follows 

In [ ]:
mask = user_activity_data_35k.outbound_activity == 'share'

expec_followers_given_share_df = compute_indirect_return(user_activity_data_35k.loc[mask], share_follower_lists)
indirect_return_share = compute_stats(expec_followers_given_share_df)

In [ ]:
indirect_return_share

In [ ]:
mask = user_activity_data_35k.outbound_activity == 'like'

expec_followers_given_like_df = compute_indirect_return(user_activity_data_35k.loc[mask], action_follower_lists)
indirect_return_like = compute_stats(expec_followers_given_like_df)

In [ ]:
indirect_return_like

In [ ]:
mask = user_activity_data_35k.outbound_activity == 'comment'

expec_followers_given_comment_df = compute_indirect_return(user_activity_data_35k.loc[mask], action_follower_lists)
indirect_return_comment = compute_stats(expec_followers_given_comment_df)

In [ ]:
indirect_return_comment

In [ ]:
follows_received_alters_succ = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/follows_received_by_alters_of_successful_creators.pkl')

def get_partitions(start_date, end_date, interval = 7):
    partitions = []
    current_date = start_date

    while current_date < end_date:
        partitions.append(current_date)
        current_date += timedelta(days=interval)

    return partitions

In [ ]:
#df_j previo
def compute_indirect_returns_week(df_j, week_start, fan_followers):
        array1 = df_j.loc[(week_start <= df_j.date_sent) & (df_j.date_sent <= week_start 
                    + timedelta(days=7))]
        if array1.shape[0] != 0:
            return np.sum(np.in1d(array1.fan_id.values, fan_followers))
        else:
            return 0

In [ ]:
follows_received['user_type'] = follows_received.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
follows_received['creator_type'] = follows_received.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

In [ ]:
follows_received_filtered = follows_received.loc[(follows_received.creator_type == 'successful')
                                                 &(follows_received.user_type != 'other')]

In [ ]:
#data should be 
def followers_up_to(user_id, date, data):
    data = data.loc[(data.contact_id==user_id)&(data.created_at <= date)].groupby('contact_id').fan_id.apply(np.array)
    return data

In [ ]:
# def compute_indirect_return_v2(data, follower_lists, follows_received): 
    
#     indirect_return_df = data[['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])
#     follows_received['date_sent'] = follows_received['date_sent'].dt.normalize()
    
#     follows_received = follows_received.loc[follows_received.user_id.isin(successful_ids)]
#     follows_received = follows_received.loc[follows_received.fan_id.isin(follower_lists.index.unique())]
#     data = data.loc[data.user_id.isin(creators.user_id)]
#     #data = data.loc[data.fan_id.isin(follower_lists.index.unique())]
#     for user_id in tqdm(follows_received.user_id.unique()):
#         indirect_return_list = []
#         df_j = data.loc[data.user_id == user_id]
        
#         for fan_id in follows_received.loc[follows_received.user_id == user_id].fan_id.unique():
#             fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]
            
#             start_date = follows_received.loc[(follows_received.user_id == user_id)&
#                                               (follows_received.fan_id == fan_id)].date_sent.iloc[0]
#             end_date = datetime(2015, 5, 30)
            
#             for week_start in get_partitions(start_date, end_date):
#                 indirect_return_list.append(
#                     compute_indirect_returns_week(df_j, week_start, fan_followers)
#                 )
                
#             indirect_return_df.loc[(indirect_return_df['user_id'] == user_id)
#                                    & (indirect_return_df['fan_id'] == fan_id), 'Indirect_return'] =\
#             sum(indirect_return_list)
                                 
#     return data

In [ ]:
def compute_indirect_return_v2(data, follower_lists, follows_received): 
    
    indirect_return_df = data[['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])
    follows_received['date_sent'] = follows_received['date_sent'].dt.normalize()
    end_date = datetime(2015, 5, 30)
    
    follows_received = follows_received.loc[follows_received.user_id.isin(successful_ids)]
    follows_received = follows_received.loc[follows_received.fan_id.isin(follower_lists.index.unique())]
    data = data.loc[data.user_id.isin(creators.user_id)]
    #data = data.loc[data.fan_id.isin(follower_lists.index.unique())]
    for user_id in tqdm(follows_received.user_id.unique()):
        indirect_return_list = []
        df_j = data.loc[data.user_id == user_id]
        
        for fan_id in tqdm(follows_received.loc[follows_received.user_id == user_id].fan_id.unique()):
            
            start_date = follows_received.loc[(follows_received.user_id == user_id)&
                                              (follows_received.fan_id == fan_id)].date_sent.iloc[0]
            
            for week_start in get_partitions(start_date, end_date):
                #print(fan_id)
                #print(week_start)
                try:
                    fan_followers = followers_up_to(fan_id, week_start
                            ,follows_received_alters_succ).values[0]
                except:
                    pass
                    
                indirect_return_list.append(
                    compute_indirect_returns_week(df_j, week_start, fan_followers)
                )
                
            indirect_return_df.loc[(indirect_return_df['user_id'] == user_id)
                                   & (indirect_return_df['fan_id'] == fan_id), 'Indirect_return'] =\
            sum(indirect_return_list)
                                 
    return data

In [ ]:
#indirect_return_follow_df = compute_indirect_return_v2(follows_received, follow_follower_lists, follows_received_filtered)

In [ ]:
indirect_return_comment

In [ ]:
!!!!! drop fan_id == user_id

In [ ]:
from joblib import Parallel, delayed

def compute_indirect_return_v2(data, follower_lists, follows_received):

    indirect_return_df = data[['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])
    follows_received['date_sent'] = follows_received['date_sent'].dt.normalize()

    follows_received = follows_received.loc[follows_received.user_id.isin(successful_ids)]
    follows_received = follows_received.loc[follows_received.fan_id.isin(follower_lists.index.unique())]
    data = data.loc[data.user_id.isin(creators.user_id)]
    # data = data.loc[data.fan_id.isin(follower_lists.index.unique())]

    def process_user(user_id):
        indirect_return_list = []
        df_j = data.loc[data.user_id == user_id]

        for fan_id in tqdm(follows_received.loc[follows_received.user_id == user_id].fan_id.unique()):
            start_date = follows_received.loc[(follows_received.user_id == user_id) &
                                              (follows_received.fan_id == fan_id)].date_sent.iloc[0]
            end_date = datetime(2015, 5, 30)

            for week_start in get_partitions(start_date, end_date):
                try:
                    fan_followers = followers_up_to(fan_id, week_start
                            ,follows_received_alters_succ).values[0]
                except:
                    pass
                try:
                    indirect_return_list.append(
                        compute_indirect_returns_week(df_j, week_start, fan_followers)
                    )
                except:
                    pass

            
            try:
                indirect_return_df.loc[(indirect_return_df['user_id'] == user_id) &
                                   (indirect_return_df['fan_id'] == fan_id), 'Indirect_return'] = \
                sum(indirect_return_list)
            except:
                pass

    Parallel(n_jobs=7)(delayed(process_user)(user_id) for user_id in tqdm(follows_received.user_id.unique()))

    return data


In [ ]:
from joblib import Parallel, delayed

def compute_indirect_return_v1(data, follower_lists, follows_received, indirect_return_df):

    follows_received['date_sent'] = follows_received['date_sent'].dt.normalize()

    follows_received = follows_received.loc[follows_received.user_id.isin(successful_ids)]
    follows_received = follows_received.loc[follows_received.fan_id.isin(follower_lists.index.unique())]
    data = data.loc[data.user_id.isin(creators.user_id)]
    # data = data.loc[data.fan_id.isin(follower_lists.index.unique())]

    def process_user(user_id):
        indirect_return_list = []
        df_j = data.loc[data.user_id == user_id]

        for fan_id in tqdm(follows_received.loc[follows_received.user_id == user_id].fan_id.unique()):
            start_date = follows_received.loc[(follows_received.user_id == user_id) &
                                              (follows_received.fan_id == fan_id)].date_sent.iloc[0]
            end_date = datetime(2015, 5, 30)
            fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]
        
            for week_start in get_partitions(start_date, end_date):
                
                indirect_return_list.append(
                        compute_indirect_returns_week(df_j, week_start, fan_followers)
                    )

            indirect_return_df.loc[(indirect_return_df['user_id'] == user_id) &
                                   (indirect_return_df['fan_id'] == fan_id), 'Indirect_return'] = \
            sum(indirect_return_list)

    Parallel(n_jobs=5)(delayed(process_user)(user_id) for user_id in tqdm(follows_received.user_id.unique()))

    #return data


In [ ]:
def compute_indirect_return_v3(data, follower_lists, follows_received_df, indirect_return_df): 
    
    follows_received_df['date_sent'] = follows_received_df['date_sent'].dt.normalize()
    data = data.loc[data.user_id != data.fan_id]
    
    follows_received_df = follows_received_df.loc[follows_received_df.user_id.isin(successful_ids)]
    follows_received_df = follows_received_df.loc[follows_received_df.fan_id.isin(follower_lists.index.unique())]
    data = data.loc[data.user_id.isin(creators.user_id)]
    #data = data.loc[data.fan_id.isin(follower_lists.index.unique())]
    for user_id in tqdm(follows_received_df.user_id.unique()):
        df_j = data.loc[data.user_id == user_id]
        
        for fan_id in tqdm(follows_received_df.loc[follows_received_df.user_id == user_id].fan_id.unique()):
            
            indirect_return_list = []
            
            fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]
            
            start_date = follows_received_df.loc[(follows_received_df.user_id == user_id)&
                                              (follows_received_df.fan_id == fan_id)].date_sent.iloc[0]
            end_date = datetime(2015, 5, 30)
            
            for week_start in get_partitions(start_date, end_date):
                indirect_return_list.append(
                    compute_indirect_returns_week(df_j, week_start, fan_followers)
                )

                
            indirect_return_df.loc[(indirect_return_df['user_id'] == user_id)
                                   & (indirect_return_df['fan_id'] == fan_id), 'Indirect_return'] =\
            sum(indirect_return_list)

In [ ]:
indirect_return_df = follows_received.loc[(follows_received.user_id.isin(successful_ids))&
                                          (follows_received.fan_id.isin(follow_follower_lists.index.unique()))
                                          ,['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])

compute_indirect_return_v3(follows_received, follow_follower_lists, follows_received_filtered, indirect_return_df)

indirect_return_df

indirect_return_df['user_type'] = indirect_return_df.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
indirect_return_df['creator_type'] = indirect_return_df.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

indirect_return_df.dropna(inplace = True)

indirect_return_df

In [ ]:
indirect_return_df.groupby('user_type', as_index = False).agg({'Indirect_return': 'sum'})

In [ ]:
mask = user_activity_data_35k.outbound_activity == 'share'
df = user_activity_data_35k.loc[mask]

indirect_return_share_df = df.loc[(df.user_id.isin(successful_ids))&
                                          (df.fan_id.isin(share_follower_lists.index.unique()))
                                          ,['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])

compute_indirect_return_v3(df, share_follower_lists, follows_received_filtered, indirect_return_share_df)

indirect_return_share_df['user_type'] = indirect_return_share_df.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
indirect_return_share_df['creator_type'] = indirect_return_share_df.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

indirect_return_share_df.dropna(inplace = True)

indirect_return_share_df


In [ ]:
indirect_return_share_df.groupby('user_type', as_index = False).agg({'Indirect_return': 'mean'})

In [ ]:
mask = user_activity_data_35k.outbound_activity == 'like'
df = user_activity_data_35k.loc[mask]

indirect_return_like_df = df.loc[(df.user_id.isin(successful_ids))&
                                          (df.fan_id.isin(action_follower_lists.index.unique()))
                                          ,['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])

compute_indirect_return_v3(df, action_follower_lists, follows_received_filtered, indirect_return_like_df)

indirect_return_like_df['user_type'] = indirect_return_like_df.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
indirect_return_like_df['creator_type'] = indirect_return_like_df.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

indirect_return_like_df.dropna(inplace = True)

indirect_return_like_df

In [ ]:
indirect_return_like_df.groupby('user_type', as_index = False).agg({'Indirect_return': 'mean'})

In [ ]:
mask = user_activity_data_35k.outbound_activity == 'comment'
df = user_activity_data_35k.loc[mask]
df = df.loc[df.user_id != df.fan_id]

indirect_return_comment_df = df.loc[(df.user_id.isin(successful_ids))&
                                          (df.fan_id.isin(action_follower_lists.index.unique()))
                                          ,['user_id', 'fan_id']].drop_duplicates(subset=['user_id', 'fan_id'])

compute_indirect_return_v3(df, action_follower_lists, follows_received_filtered, indirect_return_comment_df)

indirect_return_comment_df['user_type'] = indirect_return_comment_df.fan_id.apply(lambda x: 'f_a' if x in f_a_ids else 
                          ('Hermit' if x in hermit_ids else
                          ('Observer' if x in observer_ids else
                          ('w_a' if x in w_a_ids else 'other'))))

## classify content creators
indirect_return_comment_df['creator_type'] = indirect_return_comment_df.user_id.apply(
                               lambda x: 'successful' if x in successful_ids else 
                               ('unsuccessful' if x in unsuccessful_ids else 'other'))

indirect_return_comment_df.dropna(inplace = True)

indirect_return_comment_df

In [ ]:
indirect_return_comment_df.groupby('user_type', as_index = False).agg({'Indirect_return': 'mean', 'median', 'std'})

In [ ]:


compute_indirect_return_v3(follows_received, follow_follower_lists, follows_received_filtered, indirect_return_df)


In [ ]:
data = follows_received
follows_received_df = follows_received
follower_lists = follow_follower_lists

follows_received_df['date_sent'] = follows_received_df['date_sent'].dt.normalize()
    
follows_received_df = follows_received_df.loc[follows_received_df.user_id.isin(successful_ids)]
follows_received_df = follows_received_df.loc[follows_received_df.fan_id.isin(follower_lists.index.unique())]
data = data.loc[data.user_id.isin(creators.user_id)]
    

In [ ]:
follows_received_df

In [ ]:
user_id = 38393304

df_j = data.loc[data.user_id == user_id]

for fan_id in follows_received_df.loc[follows_received_df.user_id == user_id].fan_id.unique():

    indirect_return_list = []

    fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]

    start_date = follows_received_df.loc[(follows_received_df.user_id == user_id)&
                                      (follows_received_df.fan_id == fan_id)].date_sent.iloc[0]
    end_date = datetime(2015, 5, 30)

    for week_start in get_partitions(start_date, end_date):
        indirect_return_list.append(
            compute_indirect_returns_week(df_j, week_start, fan_followers)
        )


    indirect_return_df.loc[(indirect_return_df['user_id'] == user_id)
                           & (indirect_return_df['fan_id'] == fan_id), 'Indirect_return'] =\
    sum(indirect_return_list)


In [ ]:
indirect_return_list = []
user_id = 38027041

fan_id = 1540436
df_j = data.loc[data.user_id == user_id]
fan_followers = follower_lists.loc[follower_lists.index == fan_id].values[0]

start_date = follows_received_df.loc[(follows_received_df.user_id == user_id)&
                                      (follows_received_df.fan_id == fan_id)].date_sent.iloc[0]
end_date = datetime(2015, 6, 30)

for week_start in get_partitions(start_date, end_date):
    
    print(week_start)
    print(compute_indirect_returns_week(df_j, week_start, fan_followers))


In [ ]:
follows_received_df.loc[(follows_received_df.user_id == user_id)&
                                      (follows_received_df.fan_id == fan_id)].date_sent.iloc[0]

In [ ]:
df_j.loc[(datetime(2013, 5, 30) <= df_j.date_sent) & (datetime(2013, 5, 30) <= week_start 
                    + timedelta(days=7))].fan_id.values

In [ ]:
date = datetime(2013, 4, 27)

a = df_j.loc[(date <= df_j.date_sent) & (date <= week_start 
                    + timedelta(days=7))].fan_id.values
a[np.in1d(a,fan_followers)]

In [ ]:
date = datetime(2013, 3, 2)

a = df_j.loc[(date <= df_j.date_sent) & (date <= week_start 
                    + timedelta(days=7))].fan_id.values
a[np.in1d(a,fan_followers)]

In [ ]:
date = datetime(2013, 3, 9)

a = df_j.loc[(date <= df_j.date_sent) & (date <= week_start 
                    + timedelta(days=7))].fan_id.values
a[np.in1d(a,fan_followers)]

In [ ]:
date = datetime(2015, 3, 20)

a = df_j.loc[(date <= df_j.date_sent) & (date <= week_start 
                    + timedelta(days=7))].fan_id.values
a[np.in1d(a,fan_followers)]

In [ ]:
follows_received.loc[(follows_received.user_id == 37918716)&(follows_received.fan_id == 2023723)]